<center><img src="https://github.com/DACSS-Spatial/data_forSpatial/raw/main/logo.png" width="700"></center>

<a target="_blank" href="https://colab.research.google.com/github/DACSS-Spatial/unaryOPS/blob/main/index.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Unary Operations on  Geo Dataframes

We will review some important unary operations for GeoDataframes (GDF). This is a basic set of tools for a social scientist, but which depends a lot on the quality of the maps you have.

Keep in mind that this basic unary operations will be used later for practical applications in the coming weeks. 

## Getting ready

The links to the our maps on GitHub are here:

In [ ]:
gitMain="https://github.com/DACSS-Spatial/data_forSpatial/raw/refs/heads/main/"
linkWorldMap=gitMain+"WORLD/worldMaps.gpkg"
linkBrazil=gitMain+"BRAZIL/brazil_5880.gpkg"
linkIndicators=gitMain+"WORLD/worldindicators.json"

Let's get some maps:

In [ ]:
import geopandas as gpd

# #world 
world_rivers=gpd.read_file(linkWorldMap,layer='rivers')
# #brazil 
brazil5880=gpd.read_file(linkBrazil,layer='country')
airports_brazil5880=gpd.read_file(linkBrazil,layer='airports')
states_brazil5880=gpd.read_file(linkBrazil,layer='states')
municipalities_brazil5880=gpd.read_file(linkBrazil,layer='municipalities')
# #some indicators
indicators=gpd.read_file(linkIndicators)




Now, let's see some important spatial operations!


## I. Filtering using Pandas inheritance

### I.1 Using **iloc** and **loc**

The condition here depends on knowing index/column positions, or index/column names.

In [ ]:
#if we have
states_brazil5880.head()

In [ ]:
# iloc for positions

states_brazil5880.iloc[:5,1:] # upto fifth row

In [ ]:
# loc for labels

states_brazil5880.loc[:5,'state_code':] # upto row with label '5'

Keep in mind that if you do not include the geometry column, you will get a DataFrame (DF) back, **not** a GDF.

In [ ]:
# GDF
type(states_brazil5880.loc[:8,'state_code':])

In [ ]:
# DF
type(states_brazil5880.loc[:8,:'state_code'])

Also remember this detail:

In [ ]:
# you lose the spatial structure when keeping ONE row!
type(states_brazil5880.loc[8,:])

In [ ]:
# you keep the spatial structure if the row index is a list
type(states_brazil5880.loc[[8],:])

### I.2 Query

In [ ]:
# complex conditions with query
condition='elevation_ft > 5000 and airport_type=="small_airport"'
airports_brazil5880.query(condition)

### I.3 Subset with "isin()"

In [ ]:
choices=['large_airport','seaplane_base']
airports_brazil5880[airports_brazil5880.airport_type.isin(choices)]

### I.4 Filtering with strings

In [ ]:
# filter with text - startswith / endswith

textCondition1=('Presi','Depu')
airports_brazil5880[airports_brazil5880.airport_name.str.startswith(textCondition1)]

In [ ]:
# filter with text - contains (more flexible)

textCondition2='Presidente|Deputado'
airports_brazil5880[airports_brazil5880.airport_name.str.contains(textCondition2)]

# notice 'Refinaria Presidente Bernardes Heliport'

### I.5 Using missing values

In [ ]:
# Filter rows where specific column has no missing values

airports_brazil5880[airports_brazil5880.elevation_ft.notna()]

In [ ]:
# Filter rows where specific column has missing values

airports_brazil5880[airports_brazil5880.elevation_ft.isna()]


## II. Filtering with geometry

### II.2 Filtering by geometry attribute

The GDF 'states_brazil5880' has only these columns:

In [ ]:
states_brazil5880.columns

But, since it is projected, you can get the area of the polygon:

In [ ]:
# This filter finds states whose area is larger than 1,000,000 km²
states_brazil5880[states_brazil5880.area > 1000000000000]

Other attributes are at your service:

| Attribute | Description | Output Type |
| :--- | :--- | :--- |
| `.geometry.length` | Calculates the length of the boundary of each feature (perimeter for polygons, total length for lines). | pandas.Series (float) |
| `.geometry.centroid` | Returns the geometric center (centroid) of each geometry. | pandas.Series (Point) |
| `.geometry.bounds` | Returns the bounding box (minimum and maximum coordinates) of each geometry. | GeoDataFrame (4 columns: minx, miny, maxx, maxy) |
| `.geometry.geom_type` | Returns the type of geometry (e.g., 'Point', 'LineString', 'Polygon', 'MultiPolygon'). | pandas.Series (string) |
| `.geometry.is_empty` | Returns True if a geometry has no points (e.g., an empty geometry). | pandas.Series (boolean) |
| `.geometry.is_valid` | Returns True if a geometry is geometrically valid (no self-intersections, etc.). | pandas.Series (boolean) |
| `.geometry.exterior` | For polygons, returns the exterior ring (boundary). | pandas.Series (LinearRings) |
| `.geometry.x` | For Point geometries only, returns the X-coordinate. | pandas.Series (float) |
| `.geometry.y` | For Point geometries only, returns the Y-coordinate. | pandas.Series (float) |

We will be using these several times later.

### II.2 Slicing with **cx**

As a GDF, you can also filter using coordinates via __cx__. 

Let me get Brazil's centroid to show you how this works:

In [ ]:
brazil5880_cen=brazil5880.centroid
brazil5880_cen

Here, I recover each coordinate values:

In [ ]:
# above I mentioned you got series, then:
mid_x,mid_y=brazil5880_cen.x[0],brazil5880_cen.y[0]
mid_x,mid_y

Let me select airports north of the centroid:

In [ ]:
airports_brazil5880.cx[:,mid_y:]

Confirming we got it right:

In [ ]:
# the viz
base=brazil5880.plot(color='yellow') 
airports_brazil5880.cx[:,mid_y:].plot(ax=base,markersize=1) 
brazil5880.centroid.plot(color='red',ax=base) 

When the GDF is made of points, the __cx__ would give clean results. Do **not** expect that when GDF is made of polygons.

Let me use it to split the states:

In [ ]:
# the north
N_brazil=states_brazil5880.cx[:,mid_y:]
# the south
S_brazil=states_brazil5880.cx[:,:mid_y]
# the west
W_brazil=states_brazil5880.cx[:mid_x,:]
# the east
E_brazil=states_brazil5880.cx[mid_x:,:]

Notice the centroid does not cut polygons:

In [ ]:
base=N_brazil.plot()
brazil5880.centroid.plot(color='red',ax=base)

In [ ]:
base=W_brazil.plot()
brazil5880.centroid.plot(color='red',ax=base)

## II. Combining geometries

Let's remember these contents:

In [ ]:
#see
municipalities_brazil5880.head(10)

Then, this is Rondônia:

In [ ]:
muniRondonia=municipalities_brazil5880[municipalities_brazil5880.state_name.isin(['Rondônia'])]

I have just subset all the municipalities, then I have several polygons:

In [ ]:
muniRondonia.plot(edgecolor='yellow')



Combining the polygons means they will become ONE. We have more than one way to achieve that.

### II.1 Unary UNION

We can combine all these polygons into one:

In [ ]:
muniRondonia.union_all()

Let's save that result:

In [ ]:
Rondonia_union=muniRondonia.union_all()

It is important to know what structure we got after any operation:

- GDF?
- GeoSeries?
- Just the Geometry?

In [ ]:
type(Rondonia_union)

For GeoPandas, I recommend to turn into a GeoDataFrame these outputs. We can turn that  _shapely_ geometry into a GeoDF like this:

In [ ]:
gpd.GeoDataFrame(geometry=[Rondonia_union]) # the recent union

Even better, you can format the result with more information:

In [ ]:
gpd.GeoDataFrame(index=[0], # one element
                 data={'state':'Rondonia'}, # the column and the value
                 crs=muniRondonia.crs, # important to avoid naive geometries
                 geometry=[Rondonia_union]) # the recent union

<a class="anchor" id="21"></a>

### II.2 Dissolve

#### a. Dissolve as Union
Using  **dissolve** is an alternative to _UNION_:

In [ ]:
muniRondonia.dissolve()

Let me save the result, and see the type :

In [ ]:
Rondonia_dissolved=muniRondonia.dissolve()

# we got?
type(Rondonia_dissolved)

You got a GDF this time:

In [ ]:
## see
Rondonia_dissolved

Some minimal changes to *Rondonia_dissolved*:

In [ ]:
# keeping what is relevant
Rondonia_dissolved.drop(columns=['municipality_name','municipality_code'],inplace=True)

# then
Rondonia_dissolved

Notice _dissolving_ returns a GDF, but also keeps the information of one the lower level units that were dissolved (municipality_name, municipality_code). _Union_all_ returned just the geometry.

#### b. Dissolve for groups

Using _dissolve()_ with no arguments returns the union of the polygons as above, AND also you get a GDF.
However, if you have a column that represents a grouping (as we do), you can dissolve by that column:

In [ ]:
# dissolving municipalities again!- but by state
municipalities_brazil5880.dissolve(by='state_name').plot(facecolor='lightgrey', edgecolor='black',linewidth=0.2)

Again, let me save this result:

In [ ]:
Brazil_munitoStates=municipalities_brazil5880.dissolve(by='state_name')

We know we have a GDF; let's see contents:

In [ ]:
Brazil_munitoStates.head()

Again, we can drop columns that came from the lower level:

In [ ]:
Brazil_munitoStates.drop(columns=['municipality_name',	'municipality_code'],inplace=True)
Brazil_munitoStates.reset_index(inplace=True)
Brazil_munitoStates.info()

#### c. Dissolve and aggregate

In **pandas**, you can aggregate data using some statistics. Let's see the map with indicators we created in a previous session:

In [ ]:
indicators.head()

You can compute the mean of the countries by region, using a DF approach like this:

In [ ]:
indicators.groupby('region').agg({'fragility':'mean'}) 

You do not see a "geometry" column. It got lost when using **groupby().agg()**.

The appropriate operation to conserve spatial information is also **dissolve**:

In [ ]:
indicatorsByRegion=indicators.dissolve(
    by="region", #groupby()
    aggfunc={"fragility": "mean"}, #agg()
    )

## see the GeoDF
indicatorsByRegion

You can request a choropleth for _indicatorsByRegion_:

In [ ]:
# You may need to install if using Colab
# !pip install mapclassify

In [ ]:
indicatorsByRegion.plot(column ='fragility',edgecolor='white',
                        figsize=(15, 10))

Keep in mind that the combining of objects via UNION_ALL and DISSOLVE are destructive, we can not undo them. We have operations like EXPLODE which works in the reverse direction (splitting) but even that function can not undo the output of UNION_ALL and DISSOLVE. Always preserve your original GDF before using these operations, as they permanently alter your data in ways that cannot be reversed. By the way, _Did you notice something wrong in this last plot?_

_____________



## III. Enveloping geometries: the convex hull

Sometimes you may have the need to create a polygon that serves as an envelope to a set of points.

For this example, let me use the large airports:

In [ ]:
large_airports=airports_brazil5880.query("airport_type=='large_airport'")
large_airports.plot()

How to create a minimum polygon that envelops those points?

In [ ]:
## you see no difference!!
large_airports.convex_hull.plot()

You saw nothing new above. The objects to be enveloped required to be **previously combined**: 

In [ ]:
# hull of the union
large_airports.union_all().convex_hull

The structure we  got is:

In [ ]:
type(large_airports.union_all().convex_hull)

Let's turn this geometry into a GDF (notice that *union_all()* forced *convex_hull* to return a _geometry_):

In [ ]:
# some pythonic
params=dict(index=[0],
            data={'hull':'Large airports'}, # the column and the value
            crs=large_airports.crs,
            geometry=[large_airports.union_all().convex_hull])

LargeAirports_hull= gpd.GeoDataFrame(**params) # the unpacking

# then

LargeAirports_hull

Let's use the GDF as a layer here:

In [ ]:

base=brazil5880.plot(facecolor='yellow')
large_airports.plot(ax=base)
LargeAirports_hull.plot(ax=base,facecolor='green',
                       edgecolor='white',alpha=0.4,
                       hatch='X')

You can get a convex hull of lines or polygons:

In [ ]:
# You can use it for dissolved polygons:
Rondonia_dissolved.convex_hull.plot()

Remember that **union_all** and **dissolve()** give different outputs:

In [ ]:
#this was a GDF
type(Rondonia_dissolved)

And this is:

In [ ]:
Rondonia_dissolved.convex_hull

In [ ]:
# what?
type(Rondonia_dissolved.convex_hull)

**Rondonia_dissolved.convex_hull** is a Geoseries. We prefer to work with GDFs:

In [ ]:
# a simple "to_frame" does the job
Rondonia_dissolved.convex_hull.to_frame()

Here, we will turn that GeoSeries into a GDF:

In [ ]:
# more details
Rondonia_hull=Rondonia_dissolved.convex_hull.to_frame()
# column '0' to 'geometry'
Rondonia_hull.rename(columns={0:"geometry"},inplace=True)
# expliciting
Rondonia_hull.set_geometry('geometry',inplace=True)
# a cell value
Rondonia_hull["name"]="Rondonia"  # or: Rondonia_hull.loc[0,"name"]="Rondonia"

# we get
Rondonia_hull

Verifying CRS:

In [ ]:
#noticed the crs was inherited
Rondonia_hull.crs

Unless you need a hull per row, you need to union/dissolve the polygons (rows) of a GeoDF, see:

In [ ]:
#original not COMBINED:
Brazil_munitoStates.plot(edgecolor="yellow")

This may not be what you want, a Hull per polygon:

In [ ]:
# hull of Non combined
Brazil_munitoStates.convex_hull.plot(edgecolor="yellow")

This may be what is needed:

In [ ]:
# dissolve and envelope
Brazil_munitoStates.dissolve().convex_hull.plot(edgecolor="yellow")

## IV. Buffering geometries

The buffer will create a polygon that follows the same shape of the original vector (line, polygon, point).

Let's see the Amazon River system:

In [ ]:
AmazonSystem=world_rivers[world_rivers.SYSTEM=='Amazon']
AmazonSystem.plot()

As this is not projected...

In [ ]:
AmazonSystem.crs.is_projected

We should reproject as buffering works with distances:

In [ ]:
AmazonSystem_5880=AmazonSystem.to_crs(5880)

Now I can use the rivers to create a buffer of 50000 meters:

In [ ]:
# 50000 at each side (radius)
AmazonSystem_5880.buffer(50000).plot(facecolor='yellow', edgecolor='black',linewidth=0.2)

The resulting buffer is:

In [ ]:
type(AmazonSystem_5880.buffer(50000))

Then:

In [ ]:
base=AmazonSystem_5880.buffer(50000).plot(facecolor='yellow',edgecolor='black',linewidth=0.2)
AmazonSystem_5880.plot(ax=base)

Notice that buffering can be customized:

In [ ]:
riv_buf_right = AmazonSystem_5880.buffer(distance = 50000, single_sided = True)
riv_buf_left = AmazonSystem_5880.buffer(distance = -25000, single_sided = True)

base =riv_buf_right.plot(color='pink')
riv_buf_left.plot(ax=base, color='purple')


_____________


# Validity of Geometries

Geometries are created in a way that some issues may appear, especially during or after these operations.

Let's check if our recent maps on states and municipalities are valid:

In [ ]:
# non valid
S_brazil[~S_brazil.is_valid]

In [ ]:
# see the invalid:
S_brazil[~S_brazil.is_valid].plot()

It is difficult to see what is wrong. Let's get some information:

In [ ]:
# what is wrong?

from shapely.validation import explain_validity, make_valid

explain_validity(S_brazil[~S_brazil.is_valid].geometry)

This is the report:

In [ ]:
explain_validity(S_brazil.geometry).str.split("[",expand=True)[0].value_counts()

There is one type of invalid geometry, but there several possibilities.

Let's use **make_valid**:

In [ ]:
S_brazil_valid=S_brazil.copy()

S_brazil_valid['geometry'] = [make_valid(row)  if not row.is_valid else row for row in S_brazil['geometry'] ]

#any invalid?
S_brazil_valid[~S_brazil_valid.is_valid]

The use of *make_valid* may output **collections** (in one row several vector types: polygon, point, line). That is not desirable:

In [ ]:
import pandas as pd
pd.Series([type(x) for x in S_brazil_valid.geometry]).value_counts()

## Buffers and Validity

The buffering process helps cleaning simple invalidities:

In [ ]:
S_brazil_valid=S_brazil.copy()

S_brazil_valid['geometry'] = S_brazil_valid['geometry'].buffer(0)

#any invalid?
S_brazil_valid[~S_brazil_valid.is_valid]

This 'buffer trick' may not always work:

In [ ]:
# previously
indicatorsByRegion.plot(column =indicatorsByRegion.index,
                        edgecolor='white',
                        figsize=(15, 10))

The worst cases seem AFRICA and EAST AND SOUTHEAST ASIA, as both show some lines that should have disappeared after the dissolving we did a while ago.

Did the dissolving process created invalid geometries?

In [ ]:
indicatorsByRegion.geometry.is_valid.value_counts()

Since we do not have invalid geometries, we know the dissolving created some gaps, so the goal is to snap the boundaries together to eliminate these microscopic gaps.

We could try the trick  of buffer(0), again:

In [ ]:
indicatorsByRegion_prjd=indicatorsByRegion.to_crs("ESRI:54052").copy()
indicatorsByRegion_prjd['geometry'] = indicatorsByRegion_prjd.buffer(0)

# previously
indicatorsByRegion_prjd.plot(column =indicatorsByRegion_prjd.index,
                        edgecolor='white',
                        figsize=(15, 10))

It did not work either. 

Let's increase the buffer just to confirm gaps are presents:

In [ ]:
indicatorsByRegion_prjd_gap=indicatorsByRegion_prjd.copy()
indicatorsByRegion_prjd_gap['geometry'] = indicatorsByRegion_prjd_gap.buffer(10)

indicatorsByRegion_prjd_gap.plot(column =indicatorsByRegion_prjd_gap.index,
                                 edgecolor='white',
                                 figsize=(15, 10))

The last version did got rid of the gaps, at least visually. Let's just check the counts in each case:

In [ ]:
[(r,len(g.geoms)) for r,g in zip(indicatorsByRegion.index,indicatorsByRegion.geometry) \
 if g.geom_type.startswith('Multi')]

In [ ]:
[(r,len(g.geoms)) for r,g in zip(indicatorsByRegion_prjd_gap.index,indicatorsByRegion_prjd_gap.geometry)\
 if g.geom_type.startswith('Multi')]

indicatorsByRegion_prjd_gap did not solve the problem in Africa, but let us know there is a really big issue in those borders (Mongolia and China).